In [1]:
library(readr)
library(readxl)
library(dplyr)
library(tidyverse)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
options(repr.plot.width = 20, repr.plot.height = 10)

In [3]:
library(fgsea)
get_gsea <- function(df) {
  df <- df %>% drop_na()
  # Ensure gene names are rownames and vector is a matrix
  gene_vec <- df$fold_change
  names(gene_vec) <- df$gene

  # Wrap into a matrix for GSVA (genes x samples)
  # Read GMT
  gmt <- gmtPathways("mapk.gmt")

  # Run ssGSEA
  ss <- fgsea(stats = gene_vec, pathways = gmt)

  # Return as data frame
  ss_df <- as.data.frame((ss))
  return(ss_df)
}


In [4]:
predicted1 <- read_csv("C32_pred_de.csv")
predicted2 <- read_csv("SK-MEL-2_pred_de.csv")
predicted1$cell_line <- 'C32'
predicted2$cell_line <- 'SK-MEL-2'
predicted <- bind_rows(predicted1, predicted2)
ground_truth1 <- read_csv("C32_real_de.csv")
ground_truth2 <- read_csv("SK-MEL-2_real_de.csv")
ground_truth1$cell_line <- 'C32'
ground_truth2$cell_line <- 'SK-MEL-2'
ground_truth <- bind_rows(ground_truth1, ground_truth2)

Rows: 6000 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): target, reference
dbl (8): feature, target_mean, reference_mean, percent_change, fold_change, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 6000 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): target, reference
dbl (8): feature, target_mean, reference_mean, percent_change, fold_change, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 6000 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): target, reference
dbl (8): feature, target_mean, reference_mean, percent_change, fold_change, ...

ℹ Use `spec

In [5]:
library(reticulate)
library(dplyr)

# Load numpy and the .npy file
np <- import("numpy")
gene_names <- np$load("/large_storage/ctc/userspace/aadduri/datasets/tahoe_19k_to_2k_names.npy", allow_pickle = TRUE)
# Convert to R character vector
gene_names <- as.character(gene_names)
# Create a data frame mapping index to gene name
gene_map <- tibble(
  feature = seq_along(gene_names) - 1,  # Python 0-based index
  gene = gene_names
)
# Join with your predicted data frame
predicted <- predicted %>%
  left_join(gene_map, by = "feature")
ground_truth<- ground_truth %>%
  left_join(gene_map, by = "feature")


In [6]:


grounds_gsea <- ground_truth %>%
  group_by(cell_line, target) %>%
  group_split() %>%
  map_dfr(function(df_group) {
    gsea_res <- get_gsea(df_group)
    gsea_res$cell_line <- unique(df_group$cell_line)
    gsea_res$target <- unique(df_group$target)
    gsea_res
  })

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (13.09% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in fgseaMultilevel(pathways = pathways, stats = stats, minSize = minSize, :
“There were 1 pathways for which P-values were not calculated properly due to unbalanced (positive and negative) gene-level statistic values. For such pathways pval, padj, NES, log2err are set to NA. You can try to increase the value of the argument nPermSimple (for example set it nPermSimple = 10000)”
Warning message in fgseaMultilevel(pathways = pathways, stats = stats, minSize = minSize, :
“For some pathways, in reality P-values are less than 1e-50. You can set the `eps` argument to zero for better estimation.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (17.1% of the list

In [7]:
predicted_gsea <- predicted %>%
  group_by(cell_line, target) %>%
  group_split() %>%
  map_dfr(function(df_group) {
    gsea_res <- get_gsea(df_group)
    gsea_res$cell_line <- unique(df_group$cell_line)
    gsea_res$target <- unique(df_group$target)
    gsea_res
  })

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (7.02% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in fgseaMultilevel(pathways = pathways, stats = stats, minSize = minSize, :
“For some of the pathways the P-values were likely overestimated. For such pathways log2err is set to NA.”
Warning message in fgseaMultilevel(pathways = pathways, stats = stats, minSize = minSize, :
“For some pathways, in reality P-values are less than 1e-50. You can set the `eps` argument to zero for better estimation.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (5.43% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in fgseaMultilevel(pathways = pathways, stats = stats, minSize = minSize, :
“For some pathw

In [8]:
merged <- merge(predicted_gsea, grounds_gsea, by = c('cell_line', 'target', 'pathway'), suffixes = c('predicted', 'truth'))
merged$cell_line <- ifelse(merged$cell_line == 'C32', 'C32 - BRAF v600E mutant Melanoma', 'SK-MEL-2 - W.T. BRAF melanoma')

In [9]:
merged <- merged %>% pivot_longer(c('NESpredicted', 'NEStruth')) %>% as.data.frame()
merged$name <- ifelse(merged$name == 'NESpredicted', 'Predicted Normalized Enrichment', 'True normalized enrichment')

In [10]:
pathway <- c('REACTOME_ONCOGENIC_MAPK_SIGNALING' )

In [ ]:
 library(dplyr)
library(ggplot2)

# Step 1: Get significant MAPK pathways

# Step 2: Plot
#We only care about the largest dose, hence the grepl
p <-ggplot(
  merged %>% filter(pathway %in% pathway, grepl('5.0', target)),
  aes(x = cell_line, y = value, color = target)
) +
  facet_grid(rows = vars(pathway), cols = vars(name)) +
  ylab('Enrichment of "REACTOME_ONCOGENIC_MAPK_SIGNALING" relative to DMSO') +
  theme_bw() +
  geom_point() +
  geom_hline(yintercept = 0, linetype = "dashed")
print(p)


The geom text and geom hline dont play well with adobe, so saving a pdf without it

In [13]:
p <-ggplot(
  merged %>% filter(pathway %in% pathway, grepl('5.0', target)),
  aes(x = cell_line, y = value, color = target)
) +
  facet_grid(rows = vars(pathway), cols = vars(name)) +
  ylab('Enrichment of "REACTOME_ONCOGENIC_MAPK_SIGNALING" relative to DMSO') +
  theme_bw() +
  geom_point() 


In [14]:
ggsave('BRAF_predicted_gene_set_nohline.pdf', p)

Saving 7 x 7 in image


Warning message:
“Removed 5 rows containing missing values or values outside the scale range
(`geom_point()`).”
